In [ ]:
#importing libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import requests
from bs4 import BeautifulSoup
from csv import writer
import numpy as np
import pandas as pd
import os

In [ ]:
#display setup for visibility
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
#url adress to scrap
page_info = 'https://europeanleague.football/live-games/220821-wroclaw-panthers-leipzig-kings'

In [ ]:
#connecting with chrome 
service = Service('./chromedriver.exe')
service.start()
driver = webdriver.Remote(service.service_url)
driver.get(page_info)
time.sleep(3)
#clicking through cookies
driver.find_element(By.XPATH,'.//*[@id="consentDialog"]/div[2]/div[2]/div/div[2]/div/div[1]/div').click()
time.sleep(3)
#closing add
driver.find_element(By.XPATH,'/html/body/div[23]/div[2]/div[2]').click()
time.sleep(3)
#moving the slider to show the drives info
driver.find_element(By.XPATH,'/html/body/div[7]/div[2]/div[6]/a/div[2]').click()

In [ ]:
# expanding details for all drives
input_element = driver.find_elements(By.XPATH,'.//*[starts-with(@id, "driveindex")]/div[2]/div[7]')
for item in input_element:
    item.click()
    time.sleep(1)
page_source = driver.page_source

In [ ]:
#connecting to BS4
soup = BeautifulSoup(page_source, 'lxml')

In [ ]:
#scrapping the data
lists_all = soup.find('div', id='drives_by_quarter_inner')

for item in lists_all:
    dfs = []
    lists_drive = lists_all.find_all('div', id=lambda x : x.startswith('driveindex') if x else False)
    for drive in lists_drive:
        elements = drive.find_all(class_="drives_plays_play")
        dane = []
        qtr = drive.find_previous_sibling("h3").text
        team = drive.find('img', class_='drives teamlogo')['name']
        drive_result = drive.find('div', class_='drives_cell drivestart').text
        drive_plays = drive.find('div', class_="drives_cell plays").find_next(text=True).strip()
        drive_yds = drive.find('div', class_="drives_cell yards").find_next(text=True).strip()
        drive_time = drive.find('div', class_="drives_cell totaltime").find_next(text=True).strip()
        columns = ['aa','play','bb','situation','dd','desc','cc']
        for element in elements:
            down = element.find('div', class_="drives_plays_play_next_step").text
            play = element.find('h4').text
            desc = element.find('div', class_="drives_plays_play_desc").text
            dane.append(element)
        
        df = pd.DataFrame(dane,columns=columns)
        df['qtr'] = qtr
        df['drive_num'] = drive['id']
        df['pos_team'] = team
        df['drive_result'] = drive_result
        df['drive_plays'] = drive_plays
        df['drive_yds'] = drive_yds
        df['drive_time'] = drive_time
        dfs.append(df)
        
dfs = pd.concat(dfs)

In [ ]:
#droping not needed collumns
dfs.drop(['aa', 'bb','cc', 'dd'], axis=1,inplace=True)

In [ ]:
#reseting and dropping index
dfs.reset_index(inplace=True)
dfs.drop(columns=['index'],inplace=True)

In [ ]:
#splitting column 'desc' into 5 columns 
dfs[['down','team2','yds_to_go','team4','field_pos']] = pd.DataFrame(dfs.desc.tolist(), index = dfs.index)

In [ ]:
#using expload on new columns which contains lists into objects 
dfs = dfs.explode('play')
dfs = dfs.explode('situation')
dfs = dfs.explode('down')
dfs = dfs.explode('yds_to_go')
dfs = dfs.explode('field_pos')

In [ ]:
#droping some not needed empty columns
dfs.drop(['desc', 'team2','team4'], axis=1,inplace=True)

In [ ]:
#getting data for 3 columns with constant data for the entire game (url) from the page_info (url)
dfs['home_team'] = page_info.split('-')[4].strip().capitalize()
dfs['away_team'] = page_info.split('-')[2].strip().capitalize()
dfs['game_date'] = page_info.split('-')[1].strip().split('/')[1].strip()

In [ ]:
#saving data to csv file
if not os.path.isfile('elf_plays_2022_scrap.csv'):
    dfs.to_csv('elf_plays_2022_scrap.csv', header='column_names')
else: # else it exists so append without writing the header
    dfs.to_csv('elf_plays_2022_scrap.csv', mode='a', header=False)